In [20]:
import numpy as np
from numba import jit, autojit, double, int32
url = 'https://s3-eu-west-1.amazonaws.com/nphc-data/d10_2blocks_sym_with_Beta.pkl.gz'
from utils.loader import load_data
cumul, Beta = load_data(url)

i,j = np.random.randint(0,cumul.dim,2)
print(i,j)
n = 1e3

In [1]:
def A_ij(Z_i,Z_j,a,b,T,L_i,L_j):
    """

    Computes the mean centered number of jumps of N^j between \tau + a and \tau + b, that is

    \frac{1}{T} \sum_{\tau \in Z^i} ( N^j_{\tau + b} - N^j_{\tau + a} - \Lambda^j (b - a) )

    """
    res = 0
    u = 0
    count = 0
    n_i = Z_i.shape[0]
    n_j = Z_j.shape[0]
    for t in range(n_i):
        tau = Z_i[t]
        if tau + a < 0: continue
        while u < n_j:
            if Z_j[u] <= tau + a:
                u += 1
            else:
                break
        if u == n_j: continue
        v = u
        while v < n_j:
            if Z_j[v] < tau + b:
                v += 1
            else:
                break
        if v < n_j:
            if u > 0:
                count += 1
                res += v-u
    if count < n_i and count > 0:
        res *= n_i * 1. / count
    res /= T
    res -= (b - a) * L_i * L_j
    return res

In [ ]:
#@autojit
#@jit(nopython=True)
@jit(double(double[:],double[:],int32,int32,double,double,double), nogil=True, nopython=True)
def A_ij_numba(Z_i,Z_j,a,b,T,L_i,L_j):
    """

    Computes the mean centered number of jumps of N^j between \tau + a and \tau + b, that is

    \frac{1}{T} \sum_{\tau \in Z^i} ( N^j_{\tau + b} - N^j_{\tau + a} - \Lambda^j (b - a) )

    """
    res = 0
    u = 0
    count = 0
    n_i = Z_i.shape[0]
    n_j = Z_j.shape[0]
    for t in range(n_i):
        tau = Z_i[t]
        if tau + a < 0: continue
        while u < n_j:
            if Z_j[u] <= tau + a:
                u += 1
            else:
                break
        if u == n_j: continue
        v = u
        while v < n_j:
            if Z_j[v] < tau + b:
                v += 1
            else:
                break
        if v < n_j:
            if u > 0:
                count += 1
                res += v-u
    if count < n_i and count > 0:
        res *= n_i * 1. / count
    res /= T
    res -= (b - a) * L_i * L_j
    return res

In [21]:
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [22]:
%%cython 

import numpy as np
cimport numpy as np
cimport cython


@cython.cdivision(True)
@cython.boundscheck(False)
@cython.wraparound(False)
cpdef double A_ij_cython(double[::1] N_i, double[::1] N_j, double a, double b, double T, double L_i, double L_j):
    #cdef np.int32_t u, v, count, n_i, n_j, t
    cdef np.intp_t u, v, count, n_i, n_j, t    
    cdef double res, tau, tau_plus_a, tau_plus_b, ni_over_count
    u = 0
    count = 0
    n_i = N_i.shape[0]
    n_j = N_j.shape[0]
    res = 0.
    for t in range(n_i):
        tau = N_i[t]
        tau_plus_a = tau + a
        if tau_plus_a < 0: continue
        tau_plus_b = tau + b
        #if tau_plus_b > T: continue
        while u < n_j:
            if N_j[u] <= tau_plus_a:
                u += 1
            else: 
                break
        if u == n_j: continue
        v = u
        while v < n_j:
            if N_j[v] < tau_plus_b:
                v += 1
            else: 
                break
        if v < n_j:
            if u > 0:
                count += 1
                res += <double>(v-u)
    if count == 0:
        return - (b - a) * L_i * L_j
    else:
        if count < n_i:
            ni_over_count = <double>n_i/<double>count
            res *= ni_over_count
        res /= T
        res -= (b - a) * L_i * L_j
        return res

/Users/massil/anaconda/envs/py3k/lib/python3.4/site-packages/IPython/utils/path.py:264: UserWarning: get_ipython_cache_dir has moved to the IPython.paths module
  warn("get_ipython_cache_dir has moved to the IPython.paths module")


In [30]:
%timeit A_ij(cumul,i,j,0,n)

100 loops, best of 3: 7.42 ms per loop


In [31]:
%timeit A_ij_numba(cumul.N[i],cumul.N[j],0,n,cumul.time,cumul.L[i],cumul.L[j])

100 loops, best of 3: 7.7 ms per loop


In [32]:
%timeit A_ij_cython(cumul.N[i],cumul.N[j],0,n,cumul.time,cumul.L[i],cumul.L[j])

100 loops, best of 3: 5.26 ms per loop


In [26]:
val_1 = A_ij(cumul,i,j,-n,n)
val_2 = A_ij_numba(cumul.N[i],cumul.N[j],-n,n,cumul.time,cumul.L[i],cumul.L[j])
val_3 = A_ij_cython(cumul.N[i],cumul.N[j],-n,n,cumul.time,cumul.L[i],cumul.L[j])

In [27]:
print(val_1)
print(val_2)
print(val_3)

0.0202014207966
0.02020142079660836
0.02020142079660836
